<img src='./img/EU-Copernicus-EUM_3Logos.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='50%'></img>

<b>

<a href="./33_ltpy_Arctic_Fires_case_study.ipynb"><< 33 - Artic fires case study </a><span style="float:right;"><a href="./342_ltpy_Ozone_hole_case_study_cams_animation.ipynb">342 - Ozone hole case study - CAMS animation >></a></span>

# 3.4 Case study - Ozone hole - Summer 2019

2019, an unusual decreased in size antarctic ozone hole was monitored. Typically, the Antarctic ozone hole starts forming every year in August, reaches a maximum size in October and closes by December. Several instruments and products, including `GOME-2`, `IASI` and `CAMS`, are able to visualize the unusual behaviour.

This notebook will show examples of the unusual ozone hole in summer 2019, seen by the the following data products on `10 September 2019`:
- [GOME-2 Level-2 Total Column Ozone](#gome2)
- [IASI Total Column Ozone](#iasi)
- [CAMS Total Column Ozone forecast](#cams_nrt)

Read more about the unusual ozone hole [here](https://scienceblog.eumetsat.int/2019/09/working-together-for-the-benefit-of-our-planet/) and [here](https://atmosphere.copernicus.eu/cams-monitors-very-unusual-2019-ozone-hole).

#### Load required libraries

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import ipynb

from netCDF4 import Dataset
import pandas as pd


from matplotlib import animation

from IPython.display import HTML
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh

#### Load helper functions

In [ ]:
from ipynb.fs.full.ltpy_functions import load_l2_data_xr, generate_masked_array, generate_xr_from_1D_vec, visualize_imshow, visualize_s5p_pcolormesh, visualize_l2, visualize_gome_mollweide


<hr>

## GOME-2 Level-2 Total Column Ozone

#### Load the GOME-2 Level-2 data for Metop-A and Metop-B

Load the `total column ozone` parameter from GOME-2 Level-2 data for both satellites, Metop-A and Metop-B. You can use the function [load_l2_data_xr](./ltpy_functions.ipynb#load_l2_data_xr). The Ozone parameter is stored as `'TOTAL_COLUMNS/O3'` within the data file.

Let's first load the data for Metop-A followed by the data from Metop-B.

In [ ]:
directory_2a = './eodata/gome2/offline/2a/2019/09/10/'
o3_a_da_mf = load_l2_data_xr(directory_2a, 'TOTAL_COLUMNS', 'O3', 'LatitudeCentre', 'LongitudeCentre', no_of_dims=1,longname='Tropospheric column density of O3', unit='DU', )
o3_a_da_mf

In [ ]:
directory_2b = './eodata/gome2/offline/2b/2019/09/10/'
o3_b_da_mf = load_l2_data_xr(directory_2b, 'TOTAL_COLUMNS', 'O3', 'LatitudeCentre', 'LongitudeCentre', no_of_dims=1,longname='Tropospheric column density of O3', unit='DU', )
o3_b_da_mf

#### Load cloud fraction information

The next step is to load the `CloudFraction` information, for both, Metop-A and Metop-B data. You can use again the function [load_l2_data_xr](./ltpy_functions.ipynb#load_l2_data_xr). As internal filepath, you can specify `CLOUD_PROPERTIES` and parameter is `CloudFraction` respectively.

In [ ]:
cloud_fraction_2a = load_l2_data_xr(directory_2a,'CLOUD_PROPERTIES','CloudFraction', 'LatitudeCentre', 'LongitudeCentre', no_of_dims=1, longname='Cloud Fraction',unit='-') 
cloud_fraction_2a

In [ ]:
cloud_fraction_2b = load_l2_data_xr(directory_2b,'CLOUD_PROPERTIES','CloudFraction', 'LatitudeCentre', 'LongitudeCentre', no_of_dims=1, longname='Cloud Fraction',unit='-') 
cloud_fraction_2b

#### Mask the data array based on the cloud fraction information

Now you can mask the Ozone data with the help of the `CloudFraction` information and the function [generate_masked_array](./ltpy_functions.ipynb#generate_masked_array).

In [ ]:
o3_b_da_masked = generate_masked_array(o3_b_da_mf, cloud_fraction_2b, 0.5, '<')
o3_b_da_mf_flagged = o3_b_da_masked[o3_b_da_masked > 0]

In [ ]:
o3_a_da_masked = generate_masked_array(o3_a_da_mf, cloud_fraction_2a, 0.5, '<')
o3_a_da_masked

#### Concatenate both data arrays 

The final step before visualizing the data is to concatenate the two masked data arrays with `xarray.concat` and the common dimension `ground_pixel`.

In [ ]:
o3_ab_da_masked = xr.concat([o3_a_da_masked, o3_b_da_masked], dim='ground_pixel')
o3_ab_da_masked

#### Visualize the data in a Mollweide projection

The function [visualize_gome_mollweide](./ltpy_functions.ipynb#visualize_gome_mollweide) allows you to visualize the final data array.

In [ ]:
visualize_gome_mollweide(o3_ab_da_masked, 1, 'jet', 20, vmin=100, vmax=650)

<br>

## <a id='iasi'></a>IASI total column ozone

#### Load the data with xarray's function `open_dataset`

Load the IASI total column ozone for both, Metop-A and Metop-B satellites.

In [ ]:
file_name_2a = './eodata/iasi/nc/2a/2019/09/IASI_FORLI_O3_metopa_20190910_v20151001.nc'
file_2a = xr.open_dataset(file_name_2a)
file_2a

In [ ]:
file_name_2b_2019 = './eodata/iasi/nc/2b/2019/09/IASI_FORLI_O3_metopb_20190910_v20151001.nc'
file_2b_2019 = xr.open_dataset(file_name_2b_2019)
file_2b_2019

#### Select the variable `ozone_total_column`  and generate a `xarray`

You can select the variable of interest. In this case the variable of interest is `ozone_total_column`. You can continue to generate a xarray with the variable selected and the associated geographic coordinates. You can use the function [generate_xr_from_1D_vec](./ltpy_functions.ipynb#generate_xr_from_1d_vec).

In [ ]:
o3_2019=file_2b_2019['ozone_total_column']

o3_da_2b = generate_xr_from_1D_vec(file_2b_2019, 'latitude', 'longitude', o3_2019, 'O3', file_2b_2019.title, 1, o3_2019.units)
o3_da_2b

In [ ]:
o3=file_2a['ozone_total_column']

o3_da_2a = generate_xr_from_1D_vec(file_2a, 'latitude', 'longitude', o3, 'O3', file_2a.title, 1, o3.units)
o3_da_2a

#### Concatenate the two data array based on the dimension `ground_pixel`

In [ ]:
o3_da_2ab = xr.concat([o3_da_2a, o3_da_2b], dim='ground_pixel')

#### Convert from molecules per metre square to Dobson Unit by multiplying by 2241

The unit of the data is `molecules per metre square`. Ozone is often expressed in `Dobson Unit`. `Molecules / metre square` can be converted to `Dobson Unit [DU]` by multiplying the values with 2241.

In [ ]:
o3_da_2ab_du = o3_da_2ab * 2241

#### Flag elevated values

Ozone values higher than 500 can be flagged out with xarray's function `where`.

In [ ]:
o3_da_2ab_du_flagged = xr.where(o3_da_2ab_du > 500, np.nan, o3_da_2ab_du)

#### Visualize the data array in an `Orthographic()` projection with the function `visualize_l2`

The final step is to visualize the flagged data.

In [ ]:
visualize_l2(o3_da_2ab_du_flagged, 1, ccrs.Orthographic(180,-90),100,600, 5,'rainbow','mol m-2','Ozone total column',set_global=True)

<br>

## <a id='cams_nrt'></a>CAMS Total Column Ozone Forecast

#### Load the CAMS forecast data with xarray's function `open_dataset`

In [ ]:
cams_fc = './eodata/cams/nrt/o3/2019/09/10/20190910_o3_fc.nc'
cams_fc_xr = xr.open_dataset(cams_fc)
cams_fc_xr

#### Select the variable `gtco3` from the dataset

In [ ]:
cams_o3 = cams_fc_xr['gtco3']
cams_o3

#### Convert the data values from `kg m**-2` to `Dobson Unit`

In [ ]:
cams_o3_du = cams_o3 / (2.1415*1e-5)
cams_o3_du

#### Visualize one time step with `visualize_s5p_pcolormesh`

In [ ]:
visualize_s5p_pcolormesh(cams_o3_du[0,:,:],cams_o3_du.longitude, cams_o3_du.latitude, ccrs.Orthographic(180,-90.),'jet','DU', cams_o3.long_name + ' '+ str(cams_o3_du.time[0].data),100,600, -180, 179.6, -90. , 90., log=False,set_global=True)


See [here](./342_ltpy_Ozone_hole_case_study_cams_animation.ipynb) the CAMS Total column ozone forecast animated over five days.

<br>

<a href="./33_ltpy_Arctic_Fires_case_study.ipynb"><< 33 - Artic fires case study </a><span style="float:right;"><a href="./342_ltpy_Ozone_hole_case_study_cams_animation.ipynb">342 - Ozone hole case study - CAMS animation >></a></span>

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/atmosphere/atmosphere">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:training@eumetsat.int>Contact</a></span></p>